In [99]:
import keras
from keras import layers
import numpy as np

In [100]:
latent_dim=1
generator_input=keras.Input(shape=(latent_dim,))

In [101]:
x=layers.Dense(2)(generator_input)

In [102]:
generator=keras.models.Model(generator_input,x)
generator.summary()

Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_28 (InputLayer)        (None, 1)                 0         
_________________________________________________________________
dense_16 (Dense)             (None, 2)                 4         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


In [103]:
dimension=2
discriminator_input=keras.Input(shape=(2,))
x=layers.Dense(1)(discriminator_input)
discriminator=keras.models.Model(discriminator_input,x)
discriminator.summary()

Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 2)                 0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [104]:
discriminator_optimizer=keras.optimizers.RMSprop(lr=.0008,clipvalue=1.0)

In [105]:
discriminator.compile(optimizer=discriminator_optimizer,loss="binary_crossentropy")

In [106]:
discriminator.trainable=False
gan_input=keras.Input(shape=(1,))
gan_output=discriminator(generator(gan_input))
gan=keras.models.Model(gan_input,gan_output)

In [107]:
gan_optimizer=keras.optimizers.RMSprop(lr=0.0004)
gan.compile(optimizer=gan_optimizer,loss="binary_crossentropy")

In [110]:
iterations=10000
batch_size=200
start=0

In [111]:
for step in range(iterations):
    random_latent_vectors=np.random.normal(size=(batch_size,latent_dim))
    generated_numbers=generator.predict(random_latent_vectors)
    stop=start+batch_size
    real_numbers=np.column_stack((range(0,1000000), range(0,1000000)))
    real_numbers=real_numbers[start:stop]
    combined_numbers=np.concatenate([generated_numbers,real_numbers])
    labels=np.concatenate([np.ones((batch_size,1)),np.zeros((batch_size,1))])
    #labels+=0.05*np.random.random(labels.shape)
    d_loss=discriminator.train_on_batch(combined_numbers,labels)
    random_latent_vectors=np.random.normal(size=(batch_size,latent_dim))
    misleading_targets=np.zeros((batch_size,1))
    a_loss=gan.train_on_batch(random_latent_vectors,misleading_targets)
    start+=batch_size
    if start>len(real_numbers)-batch_size:
        start=0
    if step%1000==0:
        gan.save_weights("gan.h5")
        print("discriminator loss:",d_loss)
        print("adversarial loss:",a_loss)
        print(generated_numbers[0],real_numbers[0])

discriminator loss: 0.12585784
adversarial loss: 2.1968622
[2.7869682 2.0386865] [0 0]


KeyboardInterrupt: 